In [5]:
"""Set the problem (minus-DTLZ# as DTLZ# or minus-WFG# as WFG# or MaF#)"""
name = 'MaF4'

"""Set the number of objectives"""
M = 3

In [7]:
"""imports"""
import numpy as np
import autograd.numpy as anp
import math
import maf
from pymoo.algorithms.LHFiDi import LHFIDI
from pymoo.optimize import minimize
from pymoo.factory import get_performance_indicator
import numpy as np
from pymoo.util.normalization import normalize as norm
import hvwfg
from pymoo.model.callback import Callback
from pymoo.model.problem import Problem
from pymoo.factory import get_problem, get_crossover, get_mutation, get_reference_directions, get_selection
import os
if os.path.exists("LHFiD-i") is False:
    os.mkdir("LHFiD-i")

class MyCallback(Callback):
    def __init__(self) -> None:
        super().__init__()
        self.data["term_pop"] = None
        self.data["term_gen"] = None
    def notify(self, algorithm):
        self.data["term_pop"] = algorithm.termination_pop
        self.data["term_gen"] = algorithm.termination_suggestion

"""Set the 31 random seeds"""
seeds = np.linspace(1,1000000000,31)
seeds = [int(row) for row in seeds]

"""Set the RVs for each objective"""
def get_ref(M):
    if M==3:
        return get_reference_directions('das-dennis',3,n_partitions=13)
    elif M==5:
        return get_reference_directions('das-dennis',5,n_partitions=6)
    elif M==10:
        return get_reference_directions("multi-layer", 
                                        get_reference_directions("das-dennis", 10, n_partitions=3, scaling=1.0), 
                                        get_reference_directions("das-dennis", 10, n_partitions=2, scaling=0.5))
    elif M==15:
        return get_reference_directions("multi-layer", 
                                        get_reference_directions("das-dennis", 15, n_partitions=2, scaling=1.0), 
                                        get_reference_directions("das-dennis", 15, n_partitions=1, scaling=0.5))

"""For minus-DTLZ and minus-WFG problems"""
class minus_dtlz(Problem):
    def __init__(self, ess=None, n_var=7, n_obj=3):
        super().__init__(n_var=n_var, n_obj=n_obj, n_constr=0, xl=0 * anp.ones(n_var), xu=1 * anp.ones(n_var))
        self.n_var = n_var
        self.n_obj = n_obj
        self.ess = ess
    def _evaluate(self, x, out, *args, **kwargs):
        prob = get_problem(self.ess, n_var=self.n_var, n_obj=self.n_obj)
        f = prob.evaluate(x, return_values_of=["F"])
        f = -1*f
        out["F"] = f
class minus_wfg(Problem):
    def __init__(self, ess=None, n_var=7, n_obj=3):
        super().__init__(n_var=n_var, n_obj=n_obj, n_constr=0, xl=0 * anp.ones(n_var), xu=np.array([2*(i+1) for i in range(n_var)]))
        self.n_var = n_var
        self.n_obj = n_obj
        self.ess = ess
    def _evaluate(self, x, out, *args, **kwargs):
        prob = get_problem(self.ess, n_var=self.n_var, n_obj=self.n_obj)
        f = prob.evaluate(x, return_values_of=["F"])
        f = -1*f
        out["F"] = f
if name[0]=='W' or name[0]=='w':
    problem = minus_wfg(ess=name,n_obj=M,n_var=2*(M-1)+20)
elif name[0]=='D' or name[0]=='d':
    problem = minus_dtlz(ess=name,n_obj=M,n_var=M-1+20)
else:
    if name == 'MaF1':
        problem = maf.MaF1(n_obj=M,n_var=M-1+20)
    elif name == 'MaF4':
        problem = maf.MaF4(n_obj=M,n_var=M-1+20)
ngen = 10000 #just an arbitrary high value, LHFiD will terminate before this generation through self-termination
crossover = get_crossover('real_sbx',prob=0.9,eta=20)
if name in ['DTLZ1', 'DTLZ3']:
    mutation = get_mutation('real_pm',prob=1/problem.n_var,eta=5)
else:
    mutation = get_mutation('real_pm',prob=1/problem.n_var,eta=20)
ref_dirs = get_ref(M)
algorithm = LHFIDI(pop_size=len(ref_dirs),ref_dirs=ref_dirs,crossover=crossover,selection=get_selection('random'),mutation=mutation,eliminate_duplicates=True)
def optimize(seed_value):
    global problem,algorithm,ngen
    res = minimize(problem,algorithm,('n_gen', ngen),callback=MyCallback(),seed=seed_value)
    df = np.array(res.algorithm.callback.data["term_gen"])
    np.save('LHFiD-i/term_gen-'+name+'-'+str(M)+'-'+str(seed_value)+'.npy',df)
    df = np.array(res.algorithm.callback.data["term_pop"])
    np.save('LHFiD-i/term_pop-'+name+'-'+str(M)+'-'+str(seed_value)+'.npy',df)

for i in range (31):
    print("Run:", i+1)
    optimize(seeds[i])
    
"""Print the average termination generation"""
gens = []
for i in range(31):
    if os.path.exists('LHFiD-i/term_gen-'+name+'-'+str(M)+'-'+str(seeds[i])+'.npy'):
        gens.append(np.load('LHFiD-i/term_gen-'+name+'-'+str(M)+'-'+str(seeds[i])+'.npy'))
print(name, "( M =", str(M), ") terminated at", int(np.mean(gens)), 'generations')
hv = []
if M == 3:
    p = 13
elif M == 5:
    p = 6
elif M == 10:
    p = 3
else:
    p = 2
if name in ['DTLZ1']:
    lower = np.array([-2203.10264]*M)
    upper = np.array([0.0]*M)
elif name in ['DTLZ3']:
    lower = np.array([-4406.20528]*M)
    upper = np.array([0.0]*M)
elif name in ['DTLZ2', 'DTLZ4']:
    lower = np.array([-6.0]*M)
    upper = np.array([0.0]*M)
elif name[0]=='W' or name[0]=='w':
    upper = np.array([-1.0]*M)
    lower = np.array([(-1-2*(i+1)) for i in range(M)])
elif name in ['MaF1']:
    lower = np.array([0.0]*M)
    upper = np.array([1.0]*M)
elif name in ['MaF4']:
    upper = np.array([(2**(i+1)) for i in range(M)])
    lower = np.array([0.0]*M)
for i in range(31):
    if os.path.exists('LHFiD-i/term_pop-'+name+'-'+str(M)+'-'+str(seeds[i])+'.npy'):
        temp = np.load('LHFiD-i/term_pop-'+name+'-'+str(M)+'-'+str(seeds[i])+'.npy', allow_pickle=True)
        temp = np.array([ind.F for ind in temp])
        temp = hvwfg.wfg(norm(temp, x_min=lower, x_max=upper), np.array([2.0]*M))
        hv.append(temp) 
print("Hypervolume for",name,":", np.median(hv))

MaF4 ( M = 3 ) terminated at 1503 generations
Hypervolume for MaF4 : 7.806458007598094
